In [1]:
import json
import numpy as np
from numpy import genfromtxt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

In [2]:
# Remember index refers to half the frame number
# i = frame_num/2
keypoints = genfromtxt('imp/videos/data/0_hpe.csv', delimiter=',')

In [3]:
with open('imp/videos/labels_0/0.json', 'r') as f:
    labels = json.load(f)

In [4]:
fps_factor = 2
serve_frames = int(60/fps_factor)
# If the whole serve is 60 frames, at which frame is there contact with the ball
labelled_frame = int(50/fps_factor)
step_size = int(20/fps_factor)
num_keypoints = 18

In [5]:
# Create input to ML model
near_sequences = []
far_sequences = []
# Could do this in a single list comprehension
for i in range(serve_frames, len(keypoints), step_size):
    near_sequences.append(keypoints[i-serve_frames:i, :2*num_keypoints])
    far_sequences.append(keypoints[i-serve_frames:i, 2*num_keypoints:])
near_sequences = np.array(near_sequences)
far_sequences = np.array(far_sequences)

In [6]:
# Make labels
# Get only the frames that contain serves
near_serve_frames = []
far_serve_frames = []
for key, value in labels.items():
    if "shot" in value.keys() and value["shot"]["name"] == "F Pend Serve":
        if value["shot"]["player"] == "near":
            near_serve_frames.append(int(key))
        else:
            far_serve_frames.append(int(key))

In [12]:
near_serve_num = 0
far_serve_num = 0
near_labels = []
far_labels = []
for i in range(len(near_sequences)):
    cur_near_label = 0
    cur_far_label = 0
    lower_bound = fps_factor*(step_size*i + labelled_frame) - step_size
    upper_bound = fps_factor*(step_size*i + labelled_frame) + step_size - 1
    if near_serve_num < len(near_serve_frames):
        cur_near_frame = near_serve_frames[near_serve_num]
        if cur_near_frame >= lower_bound and cur_near_frame < upper_bound:
            cur_near_label = 1
            near_serve_num += 1
    if far_serve_num < len(far_serve_frames):
        cur_far_frame = far_serve_frames[far_serve_num]
        if cur_far_frame >= lower_bound and cur_far_frame < upper_bound:
            cur_far_label = 1
            far_serve_num += 1
    near_labels.append(cur_near_label)
    far_labels.append(cur_far_label)
near_labels = np.array(near_labels)
far_labels = np.array(far_labels)

In [13]:
model = Sequential([
    LSTM(36),
    Dense(1, activation='sigmoid')
])

In [14]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [15]:
model.fit(x=near_sequences, y=near_labels)

6/6 [==============================] - 2s 18ms/step - loss: 0.8620 - accuracy: 0.3988
